In [50]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

In [51]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

In [52]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Emily Johnson',
  'subject': 'Biology',
  'stars': 4,
  'review': "Dr. Johnson's lectures are engaging and well-structured. She's always willing to help during office hours."},
 {'professor': 'Prof. Michael Lee',
  'subject': 'Computer Science',
  'stars': 5,
  'review': 'Brilliant professor! Makes complex topics easy to understand. His coding examples are particularly helpful.'},
 {'professor': 'Dr. Sarah Martinez',
  'subject': 'Psychology',
  'stars': 3,
  'review': 'Knowledgeable, but sometimes goes off-topic. Assignments can be unclear at times.'},
 {'professor': 'Prof. David Wilson',
  'subject': 'Mathematics',
  'stars': 4,
  'review': 'Challenging course, but Prof. Wilson is patient and explains concepts thoroughly.'},
 {'professor': 'Dr. Lisa Chen',
  'subject': 'Chemistry',
  'stars': 5,
  'review': "Exceptional teacher! Her lab demonstrations are amazing and she's always prepared for class."},
 {'professor': 'Prof. Robert Brown',
  'subject': 'History',
 

In [ ]:
processed_data = []
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input=review['review'],
        model="text-embedding-3-small",
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })

In [44]:
processed_data[0]

{'values': [0.5, 0.7, 0.3, 0.2, 0.1, 0.4, 0.6, 0.8, 0.9],
 'id': 'Dr. Emily Johnson',
 'metadata': {'review': "Dr. Johnson's lectures are engaging and well-structured. She's always willing to help during office hours.",
  'subject': 'Biology',
  'stars': 4}}

In [ ]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

In [ ]:
index.describe_index_stats()